# Création du jeu de données

Données recherchées :
- Positions et vélocités des planètes par rapport au barycentre du système solaire, sur une période la plus longue possible (ça varie d'une planète à l'autre).
- Masse de chaque planète.

| ID | Name	| Start of Interval	| End of Interval |
|----|------|-------------------|-----------------|
| 10 | Sun [Sol] | B.C. 9999-03-20 00:00 |	A.D. 9999-12-31 12:00 |
| 199 |	Mercury	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 299 |	Venus	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 399 |	Earth	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 499 |	Mars	| 1600-01-01 00:00	| 2600-01-02 00:00 |
| 599 |	Jupiter	| 1600-01-10 00:00 (1600-01-11)	| 2200-01-10 00:00 (2200-01-09) |
| 699 |	Saturn	| 1749-12-30 00:00 (1749-12-31)	| 2250-01-06 00:00 |
| 799 |	Uranus	| 1600-01-04 00:00	| 2399-12-17 00:00 |
| 899 |	Neptune	| 1600-01-04 00:00	| 2400-01-02 00:00 |

## 1. Importation des librairies

In [2]:
import pandas as pd
from astroquery.jplhorizons import Horizons
from pandas import DataFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta

## 2. Définition des planètes cibles et des intervalles de temps pour lesquels récupérer les données

In [7]:
bodies: dict[str, str] = {
    "Soleil": "10",
    "Mercure": "199",
    "Venus":   "299",
    "Terre":   "399",
    "Mars":    "499",
    "Jupiter": "599",
    "Saturne": "699",
    "Uranus":  "799",
    "Neptune": "899",
    #"Pluton": "999",
}

bodies_mass: dict[str, float] = {
    "Soleil": 1.989e30,
    "Mercure": 3.3011e23,
    "Venus":   4.8675e24,
    "Terre":   5.97237e24,
    "Mars":    6.4171e23,
    "Jupiter": 1.8982e27,
    "Saturne": 5.6834e26,
    "Uranus":  8.6810e25,
    "Neptune": 1.02413e26,
    #"Pluton": 1.303e22
}

date_format: str = "%Y-%m-%d"
min_date: str = "1749-12-31" # Date minimale à partir de laquelle on a des données pour toutes les planètes (Limité par Saturne)
max_date: datetime = datetime.strptime("2200-01-09", date_format) # Date maximale à partir de laquelle on a des données pour toutes les planètes (Limité par Jupiter)
time_step = '1d' # Intervalles de 1 jour

## 3. Récupération des données

In [11]:
all_dataframes = []

for name, target in bodies.items():
    # On va chercher les données par intervalles de 100 ans parce que sur l'intervalle complet, le nombre de lignes retournées dépasse la limite permise par l'API.
    for i in range(5):
        start_date = datetime.strptime(min_date, date_format) + relativedelta(years=i * 100)
        end_date = start_date + relativedelta(years=100, days=-1)
        if end_date > max_date:
            end_date = max_date
        start_date_str = datetime.strftime(start_date, date_format)
        end_date_str = datetime.strftime(end_date, date_format)

        horizons = Horizons(
            id=target,
            location='@0', # Barycentre du système solaire
            epochs={'start': start_date_str, 'stop': end_date_str, 'step': time_step}
        )
        vectors = horizons.vectors()

        df: DataFrame = vectors.to_pandas()

        df['body_id'] = target
        df['body_name'] = name
        df['body_mass'] = bodies_mass[name]

        columns_mask = ['body_id', 'body_name', 'body_mass', 'datetime_jd', 'datetime_str','x','y','z','vx','vy','vz']

        df = df[columns_mask].copy()

        all_dataframes.append(df)

dataset = pd.concat(all_dataframes, ignore_index=True)

dataset.to_json('body_coordinates_and_velocities_from_1749-12-31_to_2200-01-09.json', orient='records', lines=True)

print(dataset.sample(5))


        body_id body_name     body_mass  datetime_jd  \
1189410     799    Uranus  8.681000e+25    2399060.5   
395352      299     Venus  4.867500e+24    2426847.5   
640160      399     Terre  5.972370e+24    2507286.5   
773414      499      Mars  6.417100e+23    2476171.5   
661582      499      Mars  6.417100e+23    2364339.5   

                           datetime_str          x          y         z  \
1189410  A.D. 1856-Apr-21 00:00:00.0000  11.877822  15.565261 -0.096300   
395352   A.D. 1932-May-20 00:00:00.0000  -0.593888  -0.406938  0.028963   
640160   A.D. 2152-Aug-13 00:00:00.0000   0.757898  -0.667686  0.000265   
773414   A.D. 2067-Jun-05 00:00:00.0000   0.568645  -1.291019 -0.041040   
661582   A.D. 1761-Mar-29 00:00:00.0000   1.287260   0.594909 -0.019849   

               vx        vy        vz  
1189410 -0.003158  0.002203  0.000049  
395352   0.011224 -0.016822 -0.000875  
640160   0.011073  0.012846 -0.000004  
773414   0.013339  0.006813 -0.000183  
661582  -0.0